# Análisis de Red de Tiendas - RetailNow
## Proyecto de análisis de datos usando Pandas y Numpy


In [2]:
# 1. IMPORTACION DE LIBRERIAS
import pandas as pd
import numpy as np

# Configurar opciones de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
# 2. CARGA Y PROCESAMIENTO DE DATOS (PANDAS)
print("=== CARGA Y PROCESAMIENTO DE DATOS ===")

# Cargar los archivos CSV
ventas_df = pd.read_csv('ventas.csv')
inventarios_df = pd.read_csv('inventarios.csv')
satisfaccion_df = pd.read_csv('satisfaccion.csv')


# Mostrar información básica de los DataFrames
print("\nInformación del DataFrame de Ventas:")
print(ventas_df.info())
print("\nPrimeras filas de ventas:")
print(ventas_df.head())

print("\nInformación del DataFrame de Inventarios:")
print(inventarios_df.info())
print("\nPrimeras filas de inventarios:")
print(inventarios_df.head())

print("\nInformación del DataFrame de Satisfacción:")
print(satisfaccion_df.info())
print("\nPrimeras filas de satisfacción:")
print(satisfaccion_df.head())

# Limpiar datos eliminando filas con valores nulos
print("\n=== LIMPIEZA DE DATOS ===")
print(f"Filas en ventas antes de limpiar: {len(ventas_df)}")
print(f"Filas en inventarios antes de limpiar: {len(inventarios_df)}")
print(f"Filas en satisfacción antes de limpiar: {len(satisfaccion_df)}")

ventas_df_clean = ventas_df.dropna()
inventarios_df_clean = inventarios_df.dropna()
satisfaccion_df_clean = satisfaccion_df.dropna()

print(f"Filas en ventas después de limpiar: {len(ventas_df_clean)}")
print(f"Filas en inventarios después de limpiar: {len(inventarios_df_clean)}")
print(f"Filas en satisfacción después de limpiar: {len(satisfaccion_df_clean)}")


=== CARGA Y PROCESAMIENTO DE DATOS ===

Información del DataFrame de Ventas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID_Tienda         10 non-null     int64 
 1   Producto          10 non-null     object
 2   Cantidad_Vendida  10 non-null     int64 
 3   Precio_Unitario   10 non-null     int64 
 4   Fecha_Venta       10 non-null     object
dtypes: int64(3), object(2)
memory usage: 532.0+ bytes
None

Primeras filas de ventas:
   ID_Tienda    Producto  Cantidad_Vendida  Precio_Unitario Fecha_Venta
0          1  Producto A                20              100  2023-01-05
1          1  Producto B                15              200  2023-01-06
2          2  Producto A                30              100  2023-01-07
3          2  Producto C                25              300  2023-01-08
4          3  Producto A                10           

# Exploración de datos (Pandas)

In [4]:
# 3. CALCULOS DE VENTAS E INGRESOS
print("=== CALCULOS DE VENTAS E INGRESOS ===")

# Ventas totales por producto
ventas_por_producto = ventas_df_clean.groupby('Producto')['Cantidad_Vendida'].sum()
print("\nVentas totales por producto:")
print(ventas_por_producto)

# Ventas totales por tienda
ventas_por_tienda = ventas_df_clean.groupby('ID_Tienda')['Cantidad_Vendida'].sum()
print("\nVentas totales por tienda:")
print(ventas_por_tienda)

# Ingresos totales por tienda (cantidad * precio)
ventas_df_clean['Ingreso'] = ventas_df_clean['Cantidad_Vendida'] * ventas_df_clean['Precio_Unitario']
ingresos_por_tienda = ventas_df_clean.groupby('ID_Tienda')['Ingreso'].sum()
print("\nIngresos totales por tienda:")
print(ingresos_por_tienda)

# Resumen estadístico de ventas
print("\nResumen estadístico de ventas:")
print(ventas_df_clean['Cantidad_Vendida'].describe())

# Resumen estadístico de ingresos
print("\nResumen estadístico de ingresos:")
print(ventas_df_clean['Ingreso'].describe())

=== CALCULOS DE VENTAS E INGRESOS ===

Ventas totales por producto:
Producto
Producto A    85
Producto B    75
Producto C    90
Name: Cantidad_Vendida, dtype: int64

Ventas totales por tienda:
ID_Tienda
1    35
2    55
3    50
4    60
5    50
Name: Cantidad_Vendida, dtype: int64

Ingresos totales por tienda:
ID_Tienda
1     5000
2    10500
3     9000
4    13000
5    13000
Name: Ingreso, dtype: int64

Resumen estadístico de ventas:
count    10.000000
mean     25.000000
std       9.128709
min      10.000000
25%      20.000000
50%      25.000000
75%      30.000000
max      40.000000
Name: Cantidad_Vendida, dtype: float64

Resumen estadístico de ingresos:
count       10.000000
mean      5050.000000
std       3361.960407
min       1000.000000
25%       2625.000000
50%       3500.000000
75%       7875.000000
max      10500.000000
Name: Ingreso, dtype: float64


# Análisis de inventarios

In [5]:
# 4. ANALISIS DE INVENTARIOS
print("=== ANALISIS DE INVENTARIOS ===")

# Crear una copia del DataFrame de inventarios para no modificar el original
inventarios_analisis = inventarios_df_clean.copy()

# Calcular ventas totales por tienda y producto
ventas_totales = ventas_df_clean.groupby(['ID_Tienda', 'Producto'])['Cantidad_Vendida'].sum().reset_index()

# Unir inventarios con ventas para calcular rotación
inventarios_analisis = inventarios_analisis.merge(
    ventas_totales, 
    on=['ID_Tienda', 'Producto'], 
    how='left'
)

# Rellenar valores nulos con 0 (productos sin ventas)
inventarios_analisis['Cantidad_Vendida'] = inventarios_analisis['Cantidad_Vendida'].fillna(0)

# Calcular rotación de inventarios (ventas/stock)
inventarios_analisis['Rotacion_Inventario'] = (
    inventarios_analisis['Cantidad_Vendida'] / inventarios_analisis['Stock_Disponible']
)

# Calcular porcentaje de productos vendidos
inventarios_analisis['Porcentaje_Vendido'] = (
    inventarios_analisis['Cantidad_Vendida'] / inventarios_analisis['Stock_Disponible'] * 100
)

print("DataFrame de inventarios con análisis:")
print(inventarios_analisis)

# Filtrar tiendas con niveles críticos (menos del 10% vendido)
niveles_criticos = inventarios_analisis[
    inventarios_analisis['Porcentaje_Vendido'] < 10
]

print("\nTiendas con niveles críticos de inventario (< 10% vendido):")
print(niveles_criticos)

# Resumen de rotación por tienda
rotacion_por_tienda = inventarios_analisis.groupby('ID_Tienda')['Rotacion_Inventario'].mean()
print("\nRotación promedio de inventarios por tienda:")
print(rotacion_por_tienda)

=== ANALISIS DE INVENTARIOS ===
DataFrame de inventarios con análisis:
   ID_Tienda    Producto  Stock_Disponible Fecha_Actualización  \
0          1  Producto A                50          2023-01-05   
1          1  Producto B                40          2023-01-06   
2          2  Producto A                60          2023-01-07   
3          2  Producto C                45          2023-01-08   
4          3  Producto A                30          2023-01-09   
5          3  Producto B                80          2023-01-10   
6          4  Producto C                70          2023-01-11   
7          4  Producto A                50          2023-01-12   
8          5  Producto B                40          2023-01-13   
9          5  Producto C                60          2023-01-14   

   Cantidad_Vendida  Rotacion_Inventario  Porcentaje_Vendido  
0                20             0.400000           40.000000  
1                15             0.375000           37.500000  
2            

# Satisfaccion del cliente

In [7]:
# 5. ANALISIS DE SATISFACCION DEL CLIENTE
print("=== ANALISIS DE SATISFACCION DEL CLIENTE ===")

# Mostrar datos de satisfacción
print("Datos de satisfacción por tienda:")
print(satisfaccion_df_clean)

# Análisis de satisfacción vs ventas
print("\n=== SATISFACCION VS RENDIMIENTO ===")

# Unir datos de satisfacción con ventas por tienda
satisfaccion_ventas = satisfaccion_df_clean.merge(
    ventas_por_tienda.reset_index(), 
    on='ID_Tienda', 
    how='inner'
)

print("Satisfacción y ventas por tienda:")
print(satisfaccion_ventas)

# Filtrar tiendas con baja satisfacción (< 60%)
tiendas_baja_satisfaccion = satisfaccion_df_clean[
    satisfaccion_df_clean['Satisfacción_Promedio'] < 60
]

print(f"\nTiendas con baja satisfacción (< 60%): {len(tiendas_baja_satisfaccion)}")
print(tiendas_baja_satisfaccion)

# Recomendaciones para tiendas con baja satisfacción
if len(tiendas_baja_satisfaccion) > 0:
    print("\n=== RECOMENDACIONES PARA MEJORAR ===")
    for _, tienda in tiendas_baja_satisfaccion.iterrows():
        print(f"Tienda {tienda['ID_Tienda']} - Satisfacción: {tienda['Satisfacción_Promedio']}%")
        print("  • Revisar atención al cliente")
        print("  • Mejorar formación del personal")
        print("  • Optimizar tiempos de espera")
        print("  • Revisar calidad de productos")

=== ANALISIS DE SATISFACCION DEL CLIENTE ===
Datos de satisfacción por tienda:
   ID_Tienda  Satisfacción_Promedio Fecha_Evaluación
0          1                     85       2023-01-15
1          2                     90       2023-01-15
2          3                     70       2023-01-15
3          4                     65       2023-01-15
4          5                     55       2023-01-15

=== SATISFACCION VS RENDIMIENTO ===
Satisfacción y ventas por tienda:
   ID_Tienda  Satisfacción_Promedio Fecha_Evaluación  Cantidad_Vendida
0          1                     85       2023-01-15                35
1          2                     90       2023-01-15                55
2          3                     70       2023-01-15                50
3          4                     65       2023-01-15                60
4          5                     55       2023-01-15                50

Tiendas con baja satisfacción (< 60%): 1
   ID_Tienda  Satisfacción_Promedio Fecha_Evaluación
4          

# Operaciones con Numpy

In [8]:
# 6. OPERACIONES CON NUMPY
print("=== OPERACIONES CON NUMPY ===")

# Convertir columna de ventas a array de Numpy
ventas_array = ventas_df_clean['Cantidad_Vendida'].to_numpy()
print("Array de Numpy de ventas:")
print(ventas_array)

# Calcular mediana y desviación estándar con Numpy
mediana_ventas = np.median(ventas_array)
desv_est_ventas = np.std(ventas_array)

print(f"\nMediana de ventas totales: {mediana_ventas}")
print(f"Desviación estándar de ventas totales: {desv_est_ventas:.2f}")

# Generar arrays aleatorios para proyecciones de ventas futuras
np.random.seed(42)  # Semilla para resultados reproducibles

# Simular proyecciones de ventas para los próximos 3 meses
proyeccion_ventas = np.random.normal(
    loc=np.mean(ventas_array),  # Media de ventas actuales
    scale=np.std(ventas_array),  # Desviación estándar de ventas actuales
    size=(3, len(ventas_array))  # 3 meses, mismo número de productos
)

print("\nProyecciones de ventas futuras (3 meses):")
print("Forma del array:", proyeccion_ventas.shape)
print("Proyección mes 1:", proyeccion_ventas[0])
print("Proyección mes 2:", proyeccion_ventas[1])
print("Proyección mes 3:", proyeccion_ventas[2])

=== OPERACIONES CON NUMPY ===
Array de Numpy de ventas:
[20 15 30 25 10 40 35 25 20 30]

Mediana de ventas totales: 25.0
Desviación estándar de ventas totales: 8.66

Proyecciones de ventas futuras (3 meses):
Forma del array: (3, 10)
Proyección mes 1: [29.30167075 23.80259603 30.60914728 38.18982546 22.97217229 22.97231447
 38.67638416 31.64617971 20.93423255 29.69870781]
Proyección mes 2: [20.98668505 20.96666202 27.09545474  8.43050704 10.06177338 20.13044715
 16.2286252  27.72146173 17.13628083 12.76909117]
Proyección mes 3: [37.69289067 23.04471988 25.58481141 12.66131877 20.28550731 25.96061781
 15.03210322 28.25364028 19.79831636 22.47385803]
